In [19]:
import tensorflow as tf
import pickle 
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

In [21]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [20]:
## Load the trained models 

model = load_model('model.h5')

##load the encoder and scaler 
with open('label_encode.pkl', 'rb') as file : 
    label_encode = pickle.load(file)

with open('onehot.pkl', 'rb') as file : 
    onehot = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file : 
    scaler = pickle.load(file)

In [23]:
%load_ext tensorboard

In [26]:
%tensorboard --logdir logs/fit/20240918-044358

Reusing TensorBoard on port 6009 (pid 28768), started 0:00:08 ago. (Use '!kill 28768' to kill it.)

In [5]:
# Input data 
input_data = {'CreditScore':600,
              'Geography':'France', 
              'Gender': 'Male', 
              'Age':40, 
              'Tenure':3, 
              'Balance':60000,
              'NumOfProducts':2, 
              'HasCrCard':1, 
              'IsActiveMember':1,
              'EstimatedSalary':50000}

In [6]:
input_data_df = pd.DataFrame([input_data])
input_data_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
geo_encoded = onehot.transform(input_data_df[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot.get_feature_names_out())
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_data_df = pd.concat([input_data_df.reset_index(drop=True),geo_encoded_df], axis = 1)
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [13]:
## Encode my categorical variables 
input_data_df['Gender'] = label_encode.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
input_data_df = input_data_df.drop('Geography', axis =1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
## Scaling now 
input_scaled = scaler.transform(input_data_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
## Predict churn 
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step


array([[0.03439076]], dtype=float32)

In [18]:
if prediction[0][0] > 0.5: 
    print('The customer is likely to churn.')
else: 
    print(' The customer is not likely to churn')

 The customer is not likely to churn
